### Daily Leads and Partners

In [3]:
import pandas as pd
import pandasql as ps

data = [['2020-12-8', 'toyota', 0, 1], ['2020-12-8', 'toyota', 1, 0], ['2020-12-8', 'toyota', 1, 2], ['2020-12-7', 'toyota', 0, 2], ['2020-12-7', 'toyota', 0, 1], ['2020-12-8', 'honda', 1, 2], ['2020-12-8', 'honda', 2, 1], ['2020-12-7', 'honda', 0, 1], ['2020-12-7', 'honda', 1, 2], ['2020-12-7', 'honda', 2, 1]]
DailySales = pd.DataFrame(data, columns=['date_id', 'make_name', 'lead_id', 'partner_id']).astype({'date_id':'datetime64[ns]', 'make_name':'object', 'lead_id':'Int64', 'partner_id':'Int64'})
print(DailySales)

     date_id make_name  lead_id  partner_id
0 2020-12-08    toyota        0           1
1 2020-12-08    toyota        1           0
2 2020-12-08    toyota        1           2
3 2020-12-07    toyota        0           2
4 2020-12-07    toyota        0           1
5 2020-12-08     honda        1           2
6 2020-12-08     honda        2           1
7 2020-12-07     honda        0           1
8 2020-12-07     honda        1           2
9 2020-12-07     honda        2           1


In [7]:
print(DailySales.groupby(['date_id', 'make_name']).nunique().reset_index())

     date_id make_name  lead_id  partner_id
0 2020-12-07     honda        3           2
1 2020-12-07    toyota        1           2
2 2020-12-08     honda        2           2
3 2020-12-08    toyota        2           3


In [18]:
query ='''
select date(date_id) as date_id, make_name, count(distinct lead_id) as unique_leads, count(distinct partner_id) as unique_partners
from DailySales
group by date_id, make_name
'''

print(ps.sqldf(query, locals()))

      date_id make_name  unique_leads  unique_partners
0  2020-12-07     honda             3                2
1  2020-12-07    toyota             1                2
2  2020-12-08     honda             2                2
3  2020-12-08    toyota             2                3


### Actors and Directors Who Cooperated At Least Three Times

In [ ]:
data = [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 3], [1, 2, 4], [2, 1, 5], [2, 1, 6]]
ActorDirector = pd.DataFrame(data, columns=['actor_id', 'director_id', 'timestamp']).astype({'actor_id':'int64', 'director_id':'int64', 'timestamp':'int64'})
print(ActorDirector)

In [17]:
result = ActorDirector.groupby(['actor_id','director_id']).count().reset_index()
print(result[result['timestamp'] >= 3][['actor_id','director_id']])

   actor_id  director_id
0         1            1


In [23]:
query = '''
select actor_id, director_id
from ActorDirector
group by actor_id, director_id
having count(*) >= 3
'''

print(ps.sqldf(query, locals()))

   actor_id  director_id
0         1            1


### Remove Employee ID With The Unique Inentifier

In [25]:
data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
Employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
EmployeeUNI = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

print(Employees)

   id      name
0   1     Alice
1   7       Bob
2  11      Meir
3  90   Winston
4   3  Jonathan


In [26]:
print(EmployeeUNI)

   id  unique_id
0   3          1
1  11          2
2  90          3


In [50]:
pd.merge(Employees, EmployeeUNI, on = 'id', how = 'left')[['unique_id', 'name']]

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


In [53]:
query = '''
select u.unique_id, e.name
from Employees E
left join
EmployeeUNI u
on u.id = e.id
'''

print(ps.sqldf(query, locals()))

   unique_id      name
0        NaN     Alice
1        NaN       Bob
2        2.0      Meir
3        3.0   Winston
4        1.0  Jonathan
